# Udacity: Deep Learning

## L4: Text and Sequences

### Train a text embedding model

Words are really hard - there are lots of them, and most of them we never see. But usually it is the rarest words that provide the most information. This is a big problem in deep learning.

### Semantic Ambiguity

We would like to share parameters between things that are similar, e.g. "cat" and "kitty." But these words are not always similar!

Big problems:

1. we'd like to see the really rare words often enough that we can learn how to use them

2. we need a way to relate words that are similar in semantic meaning

We need lots of labeled data to do this - more labeled data than we can handle! What to do...

### Unsupervised Learning

Unsupervised learning means training without labels. There is lots and lots of text out there (e.g., the web) _if_ we can figure out what to learn from it. There is a powerful concept we may exploit - similar words appear in similar contexts.

### Embeddings

* "the **cat** purrs"
* "this **cat** hunts mice"

Also reasonable to say

* "the **kitty** purrs"
* "this **kitty** hunts mice"

<img src='Screen_Shot_2016-03-01_at_6.10.15_PM.png'>

We hope we can produce a model that predicts a word's context. A model that predicts context will have to treat "cat" and "kitty" similarly and tend to bring them closer together.

<img src='Screen_Shot_2016-03-01_at_6.12.13_PM.png'>

This helps us solve the sparsity and representation problem once we have encoded our words into vectors called _embeddings_. Now all cat-like things will be similar, etc. The model no longer has to learn new things for every way there is to use a word - we get _generalization_.

### Word2Vec

"The quick brown fox jumps over the lazy dog."

<img src='Screen_Shot_2016-03-01_at_6.16.21_PM.png'>

1. We map each word in the sentance into an embedding - initially a random one.
2. We will use the embedding to try to predict the context of the word.
3. In this model, the context is the words that are nearby (within a window). We choose a random word in a window around our original word, and that becomes our target.
4. Then, we train our model as though it were a supervised problem. We use _logistic classifiers_ to make our prediction.

This is the _skip-gram_ model - we predict the context from the word. In the _continuous bag of words (CBOW)_ model, we average (? sum?) over the context to predict the target. So these approaches are two sides of the same coin - either the context is the label we want to predict from the chosen word, or the context is the data we want to use to select a target word.

## tSNE

It would be good to see that our training was working by clustering similar words closer together. One way to do this would be to do a nearest neighbors look-up. Another way would be to reduce the dimensionality of the representation and project into two dimensions. But we need a way of doing this that preserves the neighborhood structure (things that are close in embedding space should still be close in 2D, and things that are far should remain far). This means PCA ends up not working so well, but a technique called "t-SNE" works well.

<img src='Screen_Shot_2016-03-01_at_6.24.07_PM.png'>

<img src='Screen_Shot_2016-03-01_at_6.25.40_PM.png'>

### Word2Vec Details

<img src='Screen_Shot_2016-03-01_at_6.26.33_PM.png'>

The cosine distance is generally a better measure than $L_2$ - this because the length of the embedding vector is not relevant to the classification. In fact, it is often better to normalize all embedding vectors to have unit norm.

We can also use _sampled softmax_ to make our computations easier:

<img src='Screen_Shot_2016-03-01_at_6.29.29_PM.png'>

The idea behind sampled softmax is rather than treat our target vector such that the target word has probability 1 and every other word has probability 0, we only sample a small random subset of the whole rest of the vocabulary, and pretend the other words aren't there. This makes computations much more efficient and is almost as good as using the full vector.

### Word Analogy Game Quiz

<img src='Screen_Shot_2016-03-04_at_7.52.57_AM.png'>

* kitten
* shorter

### Analogies

Saying a puppy is to a dog as what a kitten is to a cat is an example of a _semnatic analogy_. Saying taller is to tall as shorter is to short is an example of a _syntactic analogy_.

<img src='Screen_Shot_2016-03-04_at_7.59.43_AM.png'>

### Segue to Assignment 5: Word2Vec and CBOW

Run Word2Vec and use it as a refernce to train a continuous bag of words (CBOW) model.

### Sequences of Varying Length

How do we deal with models of sequences of words (as opposed to just individual words)?

### RNNs

<img src='Screen_Shot_2016-05-03_at_6.18.15_PM.png'>

If our sequence is reasonably "stationary," we can use the same model at each point in time (share weights across time rather than space). But, because it is a sequence, we want to also take into account the past. A natural way to do this is to use the previous state of the classifier as a summary, recursively. We can make this efficient by using a single model to take into account the past (rather than having many layers going back hundreds or thousands of layers).

<img src='Screen_Shot_2016-05-03_at_6.22.45_PM.png'>

This creates a network with a relatively simple repeating pattern:

<img src='Screen_Shot_2016-05-03_at_6.23.18_PM.png'>

### Backprop through time

We need to back-propagate updates all the way back through time, to the very beginning of the sequence. In practice, we go as deep as we can (going to "the beginning" is usually computationally prohibitive).

All these derivatives are applied to the same parameters. This is bad for SGD, which prefers uncorrelated updates.

<img src='Screen_Shot_2016-05-03_at_6.26.20_PM.png'>

This makes the math very unstable:

<img src='Screen_Shot_2016-05-03_at_6.27.35_PM.png'>

### Vanishing / Exploding Gradients

We will fix the exploding and vanishing gradients in different ways:

1. exploding - with a hack
2. vanishing - with an elegant, but complicated, change to the model

The hack is called _gradient clipping_. In order to prevent the gradient from exploding, we cap it:

\begin{equation}
\Delta W \leftarrow \Delta W \frac{\Delta_{max}}{\max(|\Delta W|, \Delta_{max})}
\end{equation}

<img src='Screen_Shot_2016-05-04_at_8.06.34_AM.png'>

This is "hacky," but cheap and surprisingly effective.

Vanishing gradients on the other hand, make our model forget the more distant past.

<img src='Screen_Shot_2016-05-04_at_8.07.27_AM.png'>

This is harder to fix.

### LSTM

LSTM - Long short-term memory

Conceptually, a RNN consists of a repetition of simple units. At their heart is a neural network. With LSTMs, we replace the network with a LSTM 'cell.' Most of the architecture remains the same, but the vanishing gradient problem is greatly reduced.

<img src='Screen_Shot_2016-05-04_at_8.11.12_AM.png'>

<img src='Screen_Shot_2016-05-04_at_8.11.36_AM.png'>

### Memory Cell

Forgetting about neural networks - let's suppose we want a system to have memory. We need to do three things:

1. write
2. read
3. forget

<img src='Screen_Shot_2016-05-04_at_8.13.18_AM.png'>

How can we write this idea down as a diagram?

<img src='Screen_Shot_2016-05-04_at_8.14.09_AM.png'>

We use _gates_ to decide if we will write to the memory, read from it, or forget.

### LSTM Cell

We can regard our gates as multiplication by 0 when the gate is closed, or 1 when the gate is open. Now, what if instead of binary decisions at each gate, we had continuous decisions? If the gate is at 0, no information passes through. If it is at 1, all the information passes through. As it ranges between 0 and 1, "some" of the information "passes through." If this function is differentiable, we may use it in back-propagation.

This is what an LSTM is - we replace binary gates with differentiable functions to model memory.

<img src='Screen_Shot_2016-05-04_at_8.18.30_AM.png'>

### LSTM Cell 2

The gating values for each gate are controlled by a small logistic regression on the input parameters.

<img src='Screen_Shot_2016-05-04_at_8.23.07_AM.png'>

So why does this work? The gates help the model keep its memory longer when it needs it, and forget things when it should.

### Regularization

L2 regularization works well. Dropout also works, but do not use it on the recurrent connections.

<img src='Screen_Shot_2016-05-04_at_8.28.45_AM.png'>

### Beam Search

We can use sequences to generate text - one word at a time, or one character at a time. We do this repeatedly - sample + predict over and over and make a decent sequence of whatever it is our RNN is modeling.

<img src='Screen_Shot_2016-05-04_at_8.36.08_AM.png'>

We might also sample multiple times. We go several predictions deep to form a _hypothesis_. Then we compute a joint probability over all the projected steps (words/characters). This helps us avoid making one bad choice and being stuck with it forever.

<img src='Screen_Shot_2016-05-04_at_8.37.09_AM.png'>

To keep the number of hypothesis from growing out of control, we prune aggressively with a "beam search" that only keeps the most likely $N$ hypotheses at each time step.

<img src='Screen_Shot_2016-05-04_at_8.40.58_AM.png'>

### Assignment 6: LSTMs